In [1]:
import perceval as pcvl
import sympy as sp
import numpy as np
from perceval.components import BS

In [3]:
token_qcloud = '_T_eyJhbGciOiJIUzUxMiIsImlhdCI6MTY4NTQwOTEzMywiZXhwIjoxNjg4MDAxMTMzfQ.eyJpZCI6MzU3fQ.xXxxRIeyUbM6Hked_4RgIhCwZmKUkNimwubtEBssUqnmlYxFLRPvEL8fLBTcGDSgjnnH-AgvIT3NPK4YcKbCuA'
remote_simulator = pcvl.RemoteProcessor("sim:ascella", token_qcloud)
print(remote_simulator.available_commands)

['probs']


In [4]:
source = pcvl.Source() 
QPU = pcvl.Processor("Naive",2,source)
R = 0.0
QPU.add(0, pcvl.BS(BS.r_to_theta(R)))

pcvl.pdisplay(QPU, recursive = True)

states = {
    pcvl.BasicState([1, 0]): "0",
    pcvl.BasicState([0, 1]): "1",
}

ca = pcvl.algorithm.Analyzer(QPU, states)
ca.compute(expected={"0": "0", "1": "1"})
pcvl.pdisplay(ca)

,0,1
0,0,1
1,1,0


In [6]:
specs = remote_simulator.specs
print(specs["constraints"])
print(specs["parameters"])

{'max_mode_count': 12, 'max_photon_count': 6, 'min_mode_count': 1, 'min_photon_count': 1}
{'HOM': 'indistinguishability value, using HOM model (default 1)', 'backend_name': 'name of the backend that will be used for computation (default "SLOS")', 'final_mode_number': 'number of modes of the output states. states having a photon on unused modes will be ignored. Useful when using computed circuits (default input_state.m)', 'g2': 'g2 value (default 0)', 'min_detected_photons': 'minimum number of detected photons to keep a state (default input_state.n)', 'phase_imprecision': 'imprecision on the phase shifter phases (default 0)', 'transmittance': 'probability that an emitted photon is sent to the system and is detected (default 1)'}


In [9]:
my_circuit1 = pcvl.Circuit(2)  # Creates circuit with 2 spatial modes
R = 0
my_circuit1.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit1.describe()
pcvl.pdisplay(my_circuit1)

p = pcvl.Processor("Naive", my_circuit1)
states = {
    pcvl.BasicState([1, 0]): "0",
    pcvl.BasicState([0, 1]): "1",
}

ca = pcvl.algorithm.Analyzer(p, states)
ca.compute(expected={"0": "0", "1": "1"})
pcvl.pdisplay(ca)

,0,1
0,0,1
1,1,0


In [12]:
input_state = pcvl.BasicState([1, 0])

remote_simulator.set_circuit(my_circuit1)
remote_simulator.with_input(input_state)

remote_simulator.set_parameters({
    "HOM": .95,
    "transmittance": .1,
    "g2": .01
})
remote_simulator.set_parameter("mode_post_select", 1)

In [14]:
from perceval.algorithm import Sampler
sampler = Sampler(remote_simulator)
sampler.default_job_name = "My sampling job"  # All jobs created by this sampler instance will have this custom name on the cloud

nsample = 1000
remote_job = sampler.sample_count.execute_async(nsample)

In [15]:
results = remote_job.get_results()
print(results['results'])

{
  |0,1>: 1000
}


In [ ]:
input_state = pcvl.BasicState([0, 1])

remote_simulator.set_circuit(my_circuit1)
remote_simulator.with_input(input_state)

remote_simulator.set_parameters({
    "HOM": .95,
    "transmittance": .1,
    "g2": .01
})
remote_simulator.set_parameter("mode_post_select", 1)

sampler = Sampler(remote_simulator)
sampler.default_job_name = "My sampling job"  # All jobs created by this sampler instance will have this custom name on the cloud

nsample = 1000
remote_job = sampler.sample_count.execute_async(nsample)


In [17]:
results = remote_job.get_results()
print(results['results'])

{
  |1,0>: 1000
}


In [18]:
remote_qpu = pcvl.RemoteProcessor("qpu:ascella", token_qcloud)
remote_qpu.set_circuit(my_circuit1)
remote_qpu.with_input(input_state)
remote_qpu.min_detected_photons_filter(1)

sampler_on_qpu = Sampler(remote_qpu)

nsample = 1000
remote_job = sampler_on_qpu.sample_count
remote_job.name = "QPU sampling"  # You may also specify a name to individual jobs
remote_job.execute_async(nsample)

In [20]:
results = remote_job.get_results()
print(results['results'])

{
  |1,0>: 122082
  |0,1>: 76
}


In [22]:
my_circuit2 = pcvl.Circuit(2)  # Creates circuit with 2 spatial modes
R = 0
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))

my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))

my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))
my_circuit2.add((0,1), pcvl.BS(BS.r_to_theta(R)))

my_circuit2.describe()
pcvl.pdisplay(my_circuit2)

p = pcvl.Processor("Naive", my_circuit2)
states = {
    pcvl.BasicState([1, 0]): "0",
    pcvl.BasicState([0, 1]): "1",
}

ca = pcvl.algorithm.Analyzer(p, states)
ca.compute(expected={"0": "0", "1": "1"})
pcvl.pdisplay(ca)

,0,1
0,0,1
1,1,0


In [23]:
input_state = pcvl.BasicState([1, 0])
remote_qpu = pcvl.RemoteProcessor("qpu:ascella", token_qcloud)
remote_qpu.set_circuit(my_circuit2)
remote_qpu.with_input(input_state)
remote_qpu.min_detected_photons_filter(1)

sampler_on_qpu = Sampler(remote_qpu)

nsample = 1000
remote_job = sampler_on_qpu.sample_count
remote_job.name = "QPU sampling"  # You may also specify a name to individual jobs
remote_job.execute_async(nsample)

In [24]:
results = remote_job.get_results()
print(results['results'])

{
  |1,0>: 76
  |0,1>: 113719
}
